In [1]:
import pandas as pd

In [2]:
leaderboard_data = pd.read_csv('leaderboard_data.csv')
racecourse_data = pd.read_csv('race_course_data.csv')
rider_season_data = pd.read_csv('rider_season_data.csv')
rider_data = pd.read_csv('rider_data.csv')

# changing time to gaps in seconds
def convert_to_seconds(time):
    time = time.split(':')
    if len(time) < 2:
        return -1
    elif len(time) > 2:
        return 0
    return int(time[0])*60 + int(time[1])

leaderboard_data['time'] = leaderboard_data['time'].apply(convert_to_seconds)

# drop date from racecourse data
racecourse_data = racecourse_data.drop(columns=['date', 'year', 'name', 'stage'])

# merge the data based on url, add the course data to the leaderboard data
leaderboard_course_merged = pd.merge(leaderboard_data, racecourse_data, on='url', how='inner', validate='many_to_one')

rider_data = rider_data.rename(columns={'url': 'rider_url', 'name': 'rider_name'})

def standardize_name(name):
    name = name.split()
    name.sort()
    return ' '.join(name)

rider_data['merge_name'] = rider_data['rider_name'].str.lower().apply(standardize_name)
leaderboard_course_merged['merge_name'] = leaderboard_course_merged['rider'].str.lower().apply(standardize_name)


# merge the data based on rider name, add the rider data to the leaderboard data
leaderboard_course_rider_merged = pd.merge(leaderboard_course_merged, rider_data, on='merge_name', how='inner', validate='many_to_one')
leaderboard_course_rider_merged = leaderboard_course_rider_merged.drop(columns=['rider', 'merge_name'])

In [23]:
# preprocess merged data to make number columns ready for training
merged_data = leaderboard_course_rider_merged.copy()

# change rank to a number
def convert_rank(rank):
    try:
        return int(rank)
    except:
        return -1
    
merged_data['rank'] = leaderboard_course_rider_merged['rank'].apply(convert_rank)

# change distance to a number
def convert_distance(distance):
    distance = distance.split()
    return float(distance[0])
    
merged_data['distance'] = leaderboard_course_rider_merged['distance'].apply(convert_distance)

# change speed to a number
def convert_speed(speed):
    speed = speed.split()
    if len(speed) < 2:
        return None
    return float(speed[0])

merged_data['speed'] = leaderboard_course_rider_merged['speed'].apply(convert_speed)

#change weight to a number
def convert_weight(weight):
    if isinstance(weight, float):
        return weight
    weight = weight.split()
    return float(weight[0])

merged_data['weight'] = leaderboard_course_rider_merged['weight'].apply(convert_weight)

# change height to a number
def convert_height(height):
    if isinstance(height, float):
        return height
    height = height.split()
    return float(height[0])

merged_data['height'] = leaderboard_course_rider_merged['height'].apply(convert_height)

# if won includes solo, than just set it to solo
def convert_won(won):
    if 'solo' in won:
        return 'Solo'
    elif 'Sprint' in won and 'riders' in won:
        if int(won.split()[2]) < 11:
            return 'Sprint of small group'
        else:
            return 'Sprint of large group'
    elif '?' in won:
        return 'Other'
    return won[1:]

merged_data['won'] = leaderboard_course_rider_merged['won'].apply(convert_won)

def merge_name_stage(row):
    return row['name'] + ' ' + row['stage']

merged_data['name'] = merged_data.apply(merge_name_stage, axis=1)
merged_data = merged_data.drop(columns=['rider_url', 'url', 'stage', 'ranking'])

print(merged_data.columns)

merged_data.to_csv('merged_data.csv', index=False)

Index(['year', 'name', 'rank', 'age', 'speciality', 'team', 'time', 'distance',
       'speed', 'score', 'vertical_meters', 'quality', 'won', 'rider_name',
       'nationality', 'weight', 'height', 'one_day', 'gc', 'tt', 'sprint',
       'climber', 'hills'],
      dtype='object')
